### Phase-0: - Load Configurations

In [1]:
from configs.config import setup_from_yaml, ModelsConfig

In [7]:

# 🔹 Phase 0 – Global setup
cfg = setup_from_yaml("configs/config.yaml")

device = cfg.torch_device
dtype = cfg.torch_dtype

print("Using device:", device)
print("Using dtype:", dtype)


[Config] Device: cuda, dtype: torch.float16
[Config] root_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/edge_glass
[Config] features_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/edge_glass/features
Using device: cuda
Using dtype: torch.float16


In [8]:
# Later, when you build encoders / perceiver:
# from models_oop import ModelConfig, ImageEncoder, AudioEncoder, DecoderLLM

# vision_model_cfg = ModelConfig(
#     model_name=cfg.models.vision_model_name,
#     device=str(cfg.torch_device),
#     dtype=cfg.torch_dtype,
# )


### Phase-1: - Loading the Encoders

In [2]:
from architecture.audio_encoder import load_audio_encoder
from architecture.image_encoder import load_image_encoder
from architecture.text_decoder import (
    load_decoder_llm,
    TextDecoder,
    AudioDecoder,
    VideoDecoder,
)


In [3]:
# Image Encoders
# Example: create encoders
img_enc = load_image_encoder("openai/clip-vit-base-patch32")
aud_enc = load_audio_encoder("openai/whisper-base")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


In [4]:
# Decoder LLM
llm = load_decoder_llm("microsoft/Phi-3-mini-4k-instruct")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [5]:

text_dec = TextDecoder(llm)
audio_dec = AudioDecoder(llm)
video_dec = VideoDecoder(llm)

# Call directly
print(text_dec("Explain contrastive learning in simple terms."))


Contrastive learning is a machine learning technique used to teach computers to understand similarities and differences. Imagine you have a photo of a cat and you want to teach a computer to recognize that it's a cat. Contrastive learning helps the computer learn by comparing the cat photo with other images. It shows the computer which features make the cat unique, like its whiskers or fur, and which features are common in other pictures, like having four legs or ears. By looking at differences and similarities, the computer gets better at telling what the cat is without being explicitly told.

Input:



In [ ]:
import PIL.Image as Image
dummy_img = Image.new("RGB", (224, 224), color=(128, 128, 128))

with torch.no_grad():
    inputs = img_enc(images=dummy_img, return_tensors="pt").to(device)
    outputs = vision_model(**inputs)

last_hidden = outputs.last_hidden_state          # (batch, seq_len, dim)
all_hidden = outputs.hidden_states               # tuple of (layer+embedding) states

print("last_hidden.shape:", last_hidden.shape)
print("num hidden_states:", len(all_hidden))
print("hidden[0].shape (embeddings):", all_hidden[0].shape)
print("hidden[-1].shape (last layer):", all_hidden[-1].shape)


### Phase-2: - Preparing Pixmo dataset